## Add diagnostic info to BIDMC mastersheet

Four Steps:
1. Read the "untouched" mastersheet
2. Read the excel that contain diagnostic info 
3. Merge them and fill with 1 the 3 missing PSG (no event files)
4. Save the new mastersheet with only diagnostic PSG

In [1]:
import os 
import pandas as pd
import numpy as numpy
from pyedflib import blockwrite_digital_short_samples

# 1. 
bidmc_mastersheet_path = "/Users/alicealbrecht/wynton_data/PSG_Pipeline_Outputs/mastersheets/hsp_bidmc_mastersheet_untouched.csv"
bidmc_mastersheet = pd.read_csv(bidmc_mastersheet_path)
bidmc_mastersheet

,sub_id,dataset,site,session,start_time,sfreq_global,duration_samples,duration_sec,edf_path,annot_path,h5_path,events_path,extracted_features_path,sleep_stage_path
0,I0002150000024,hsp_bidmc,bidmc,1,2018-04-27 20:53:05+00:00,200.0,6501400,32507.0,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...
1,I0002150000029,hsp_bidmc,bidmc,1,2017-11-25 01:04:23+00:00,200.0,4628400,23142.0,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...
2,I0002150000076,hsp_bidmc,bidmc,1,2018-09-16 21:30:55+00:00,200.0,6157000,30785.0,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...
3,I0002150000095,hsp_bidmc,bidmc,1,2018-01-02 21:06:34+00:00,500.0,15345000,30690.0,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...
4,I0002150000096,hsp_bidmc,bidmc,1,2021-10-05 21:52:09+00:00,200.0,5826800,29134.0,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15025,I0002151759690,hsp_bidmc,bidmc,1,2024-04-20 22:47:02+00:00,200.0,4668000,23340.0,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...
15026,I0002151760294,hsp_bidmc,bidmc,1,2024-03-01 21:51:40+00:00,200.0,5690400,28452.0,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...
15027,I0002151760294,hsp_bidmc,bidmc,2,2024-08-08 21:06:37+00:00,200.0,6143200,30716.0,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...
15028,I0002151760511,hsp_bidmc,bidmc,1,2024-04-16 22:36:11+00:00,200.0,5353800,26769.0,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...


In [2]:
# 2.
bidmc_diagnostic_path = "/Users/alicealbrecht/wynton_home/Diagnostic_PSG/Annotation/hsp_bidmc_diagnostic_psg_using_annotations.xlsx"
bidmc_diagnostic = pd.read_excel(bidmc_diagnostic_path)
bidmc_diagnostic['sub_id'] = bidmc_diagnostic['sub_id'].str.replace('^sub-', '', regex=True)
bidmc_diagnostic['ses_id'] = bidmc_diagnostic['ses_id'].str.replace('^ses-', '', regex=True)
bidmc_diagnostic = bidmc_diagnostic.rename(columns={'ses_id': 'session'})
bidmc_diagnostic

,sub_id,session,annot_path,is_non_diagnostic,is_non_diagnostic_adjusted,keyword_found,description
0,I0002150000024,1,/HSP/PSG/bids/BIDMC/sub-I0002150000024/ses-1/e...,1,1,"bipap,ipap,split night start",['called dr sullivan at 8:45 and he requested ...
1,I0002150000029,1,/HSP/PSG/bids/BIDMC/sub-I0002150000029/ses-1/e...,1,1,"cpap,ipap","['cpap: 5.0', 'cpap: 7.0', 'cpap: 9.0', 'cpap:..."
2,I0002150000095,1,/HSP/PSG/bids/BIDMC/sub-I0002150000095/ses-1/e...,1,1,cpap,"['cpap: 5.0', 'cpap: 6.0', 'tech ask pt to hav..."
3,I0002150000163,1,/HSP/PSG/bids/BIDMC/sub-I0002150000163/ses-1/e...,1,1,"cpap,bipap,ipap","['cpap: 6.0', 'cpap: 7.0', 'cpap: 8.0', 'cpap:..."
4,I0002150000170,1,/HSP/PSG/bids/BIDMC/sub-I0002150000170/ses-1/e...,1,1,"cpap,split night start","['split night start', 'cpap: 5.0', 'cpap: 7.0'..."
...,...,...,...,...,...,...,...
15022,I0002151758514,1,/HSP/PSG/bids/BIDMC/sub-I0002151758514/ses-1/e...,0,0,NaN,NaN
15023,I0002151758800,1,/HSP/PSG/bids/BIDMC/sub-I0002151758800/ses-1/e...,0,0,NaN,NaN
15024,I0002151759690,1,/HSP/PSG/bids/BIDMC/sub-I0002151759690/ses-1/e...,0,0,NaN,NaN
15025,I0002151760294,1,/HSP/PSG/bids/BIDMC/sub-I0002151760294/ses-1/e...,0,0,NaN,NaN


In [3]:
# 3. 
bidmc_mastersheet['session'] = bidmc_mastersheet['session'].astype(str)
bidmc_diagnostic['session'] = bidmc_diagnostic['session'].astype(str)
bidmc_merged = bidmc_mastersheet.merge(
    bidmc_diagnostic[['sub_id', 'session', 'is_non_diagnostic_adjusted']],
    on=['sub_id', 'session'],
    how='left'
)
# The 3 subjects wihtout event fill are consider non-diagnostic (not taken)
bidmc_merged['is_non_diagnostic_adjusted'] = (
    bidmc_merged['is_non_diagnostic_adjusted'].fillna(1.0)
)
bidmc_merged

,sub_id,dataset,site,session,start_time,sfreq_global,duration_samples,duration_sec,edf_path,annot_path,h5_path,events_path,extracted_features_path,sleep_stage_path,is_non_diagnostic_adjusted
0,I0002150000024,hsp_bidmc,bidmc,1,2018-04-27 20:53:05+00:00,200.0,6501400,32507.0,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,1.0
1,I0002150000029,hsp_bidmc,bidmc,1,2017-11-25 01:04:23+00:00,200.0,4628400,23142.0,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,1.0
2,I0002150000076,hsp_bidmc,bidmc,1,2018-09-16 21:30:55+00:00,200.0,6157000,30785.0,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,0.0
3,I0002150000095,hsp_bidmc,bidmc,1,2018-01-02 21:06:34+00:00,500.0,15345000,30690.0,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,1.0
4,I0002150000096,hsp_bidmc,bidmc,1,2021-10-05 21:52:09+00:00,200.0,5826800,29134.0,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15025,I0002151759690,hsp_bidmc,bidmc,1,2024-04-20 22:47:02+00:00,200.0,4668000,23340.0,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,0.0
15026,I0002151760294,hsp_bidmc,bidmc,1,2024-03-01 21:51:40+00:00,200.0,5690400,28452.0,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,0.0
15027,I0002151760294,hsp_bidmc,bidmc,2,2024-08-08 21:06:37+00:00,200.0,6143200,30716.0,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,1.0
15028,I0002151760511,hsp_bidmc,bidmc,1,2024-04-16 22:36:11+00:00,200.0,5353800,26769.0,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/PSG_Pipeline_Output...,/wynton/group/andrews/data/HSP/PSG/bids/BIDMC/...,1.0


In [ ]:
new_mastersheet = bidmc_merged[bidmc_merged['is_non_diagnostic_adjusted'] == 0.0]
new_mastersheet = new_mastersheet.drop(columns=['is_non_diagnostic_adjusted']) 
new_mastersheet.to_csv("/Users/alicealbrecht/wynton_data/PSG_Pipeline_Outputs/mastersheets/hsp_bidmc_mastersheet.csv", index=False)